In [27]:
import torch
import torch.nn as nn
import pandas as pd

In [28]:
class CFFN(nn.Module):
    def __init__(self):
        super(CFFN, self).__init__()
        self.hidden = nn.Linear(3, 20)
        self.relu = nn.ReLU()
        self.out_from_hidden = nn.Linear(20, 1)
        self.out_from_source = nn.Linear(3, 1)

    def forward(self, x):
        out = self.hidden(x)
        out = self.relu(out)
        out_h = self.out_from_hidden(out)
        out_s = self.out_from_source(x)
        out_final = out_s + out_h
        return out_final

In [29]:
# Parameters
hidden_neurons = 20  # Use same as in training
model_path = "trained_cffn.pth"
csv_input_path = "../flight_data_test.csv"
csv_output_path = "predicted_output.csv"

In [30]:

# 1. Load input CSV
df = pd.read_csv(csv_input_path)
input_tensor = torch.tensor(df.values, dtype=torch.float32)  # shape: (N, 3)


In [31]:

# 2. Load model and weights
model = CFFN()
model.load_state_dict(torch.load(model_path))
model.eval()

CFFN(
  (hidden): Linear(in_features=3, out_features=20, bias=True)
  (relu): ReLU()
  (out_from_hidden): Linear(in_features=20, out_features=1, bias=True)
  (out_from_source): Linear(in_features=3, out_features=1, bias=True)
)

In [32]:
# 3. Predict
with torch.no_grad():
    predictions = model(input_tensor)  # shape: (N, 3)

In [33]:
# 4. Save predictions to CSV
pred_array = predictions.numpy()
df_pred = pd.DataFrame(pred_array, columns=["Output1"])
df_combined = pd.concat([df, df_pred], axis=1)

df_combined.to_csv(csv_output_path, index=False)
print(f"Predictions saved to {csv_output_path}")

Predictions saved to predicted_output.csv
